In [1]:
#!pip install -U weaviate-client
!pip install "weaviate-client==3.*" # Version 4 of the client does not work

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 8.9 MB/s eta 0:00:00


In [2]:
import weaviate

WCS_ENDPOINT = "https://my-sandbox1-1486fdzz.weaviate.network/"
WCS_API_KEY = "YOUR_API_KEY"
OPENAI_API_KEY = 'YOUR_API_KEY'

client = weaviate.Client(
    url=WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
)

client.schema.get()

/opt/conda/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.2.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


{'classes': [{'class': 'MyTable1',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {},
    'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Mon Feb  5 15:13:09 2024",
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'title',
     'tokenization': 'word'},
    {'dataType': ['number'],
     'description': "This property was generated by Weaviate's auto-schema feature on Mon Feb  5 15:13:09 2024",
     'indexFilterable': True,
     'indexSearchable': False,
     'moduleConfig': {'te

In [3]:
import weaviate, json
from weaviate import EmbeddedOptions

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
)

client.is_ready()

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.0/weaviate-v1.23.0-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 81


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-06T02:19:58Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-06T02:19:58Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-06T02:19:58Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-02-06T02:19:58Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-02-06T02:19:58Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"202

True

In [4]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("MyCollection"):
     client.schema.delete_class("MyCollection")

schema = {
    "class": "MyCollection",
    "vectorizer": "none",
    "vectorIndexConfig": {
        "distance": "cosine" # let's use cosine distance
    },
}

client.schema.create_class(schema)

print("Successfully created the schema.")

Successfully created the schema.


{"level":"info","msg":"Created shard mycollection_F7impXisod5H in 1.10156ms","time":"2024-02-06T02:19:58Z"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-06T02:19:58Z","took":198190}


## [ 1 ] Insert my own vectors into the database

In [5]:
# Dummy data

data = [
   {
      "title": "First Object",
      "foo": 99, 
      "vector": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
   },
   {
      "title": "Second Object",
      "foo": 77, 
      "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
   },
   {
      "title": "Third Object",
      "foo": 55, 
      "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
   },
   {
      "title": "Fourth Object",
      "foo": 33, 
      "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
   },
   {
      "title": "Fifth Object",
      "foo": 11,
      "vector": [0.5, 0.5, 0, 0, 0, 0]
   },
]

In [6]:
client.batch.configure(batch_size=10)  # Configure batch

# Batch import all objects
# yes batch is an overkill for 5 objects, but it is recommended for large volumes of data
with client.batch as batch:
  for item in data:

    # This is the meta data
    properties = {
     "title": item["title"],
     "foo": item["foo"],
    }

    # the call that performs data insert
    client.batch.add_data_object(
     class_name="MyCollection",
     data_object=properties,
     vector=item["vector"] # your vector embeddings go here
    )


In [7]:
# Check number of objects
response = (
    client.query
    .aggregate("MyCollection")
    .with_meta_count()
    .do()
)

print(response)

{'data': {'Aggregate': {'MyCollection': [{'meta': {'count': 5}}]}}}


## Run a similarity search

In [8]:
response = (
    client.query
    .get("MyCollection", ["title"]) # We want the response to contain the title of each vector
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5] # This is the query vector
    })
    .with_limit(2) # limit the output to only 2
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))

[
  {
    "title": "Second Object"
  },
  {
    "title": "Fourth Object"
  }
]


In [9]:
# Display the embeddings for the matched objects and the cosine similarity

response = (
    client.query
    .get("MyCollection", ["title"])
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
    })
    .with_limit(2) # limit the output to only 2
    .with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))

[
  {
    "_additional": {
      "distance": 0.6506307,
      "id": "95862fe6-d8f7-414c-9101-b105fba242b7",
      "vector": [
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
      ]
    },
    "title": "Second Object"
  },
  {
    "_additional": {
      "distance": 0.8072029,
      "id": "daf32ffd-79fe-455d-8126-8465cf0b3bf2",
      "vector": [
        0.4,
        0.41,
        0.42,
        0.43,
        0.44,
        0.45
      ]
    },
    "title": "Fourth Object"
  }
]


## [ 2 ] Upload text and let Weaviate create the vectors

In [10]:
"The quick brown fox jumps over the lazy dog."
"A nimble red fox leaped over the sleeping hound."
"In the heart of winter, I discovered an unbeatable summer within me."
"During the coldest season, there lies an invincible summer inside us."
"Technology connects people in unprecedented ways."

"Innovations in technology foster human connections."
"Every long journey begins with a single step forward."
"A thousand-mile trek starts with one step."
"Art is the illusion that reveals the truth."
"Through art, we tell lies that help us understand the truth."

'Through art, we tell lies that help us understand the truth.'

In [11]:
#qqq

In [12]:
# Dummy data

# Note: It looks like all the properties get vectorized.
# It's possible to indicate that a propery should not be vectorized.
# It looks like properties that are not text don't get vectorized.
# Ref: https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules

data = [
   {
      "title": "Object0",
      "foo": 99, 
      "quote_text": "The quick brown fox jumps over the lazy dog."
   },
   {
      "title": "Object1",
      "foo": 77, 
      "quote_text": "A nimble red fox leaped over the sleeping hound."
   },
   {
      "title": "Object2",
      "foo": 55, 
      "quote_text": "In the heart of winter, I discovered an unbeatable summer within me."
   },
   {
      "title": "Object3",
      "foo": 33, 
      "quote_text": "During the coldest season, there lies an invincible summer inside us."
   },
   {
      "title": "Object4",
      "foo": 11,
      "quote_text": "Technology connects people in unprecedented ways."
   },  
    {
      "title": "Object5",
      "foo": 99, 
      "quote_text": "Innovations in technology foster human connections."
   },
   {
      "title": "Object6",
      "foo": 77, 
      "quote_text": "Every long journey begins with a single step forward."
   },
   {
      "title": "Object7",
      "foo": 55, 
      "quote_text": "A thousand-mile trek starts with one step."
   },
   {
      "title": "Object8",
      "foo": 33, 
      "quote_text": "Art is the illusion that reveals the truth."
   },
   {
      "title": "Object9",
      "foo": 11,
      "quote_text": "Through art, we tell lies that help us understand the truth."
   },
]

In [13]:
import weaviate
import json

client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

In [14]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("MyTable1"):
     client.schema.delete_class("MyTable1")


# Create a Table in the database.
# We call this table a "class".

class_obj = {
    "class": "MyTable1", # This is the name of the database table
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

In [15]:
import requests
import json

client.batch.configure(batch_size=5)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, data in enumerate(data):  # Batch import data
        print(f"importing question: {i+1}")
        properties = {
            "title": data["title"],
            "foo": data["foo"],
            "quote_text": data["quote_text"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="MyTable1"
        )

{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2024-02-06T02:19:59Z"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-02-06T02:19:59Z"}


importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [16]:
# Check number of objects in MyTable1

response = (
    client.query
    .aggregate("MyTable1")
    .with_meta_count()
    .do()
)

print(response)

{'data': {'Aggregate': {'MyTable1': [{'meta': {'count': 10}}]}}}


In [17]:
# Run a similarity search

import weaviate
import json

client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

query_text = "What did the fox do?"

response = (
    client.query
    .get("MyTable1", ["title", "foo", "quote_text"])
    .with_near_text({"concepts": [query_text]})
    .with_limit(2)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "MyTable1": [
                {
                    "_additional": {
                        "distance": 0.15643907,
                        "id": "d81aa2a5-ab34-41e3-b275-bf2bfe7eeb8b"
                    },
                    "foo": 77,
                    "quote_text": "A nimble red fox leaped over the sleeping hound.",
                    "title": "Object1"
                },
                {
                    "_additional": {
                        "distance": 0.20395815,
                        "id": "f7c132b8-422b-4b76-aff6-d066e2c58de1"
                    },
                    "foo": 99,
                    "quote_text": "The quick brown fox jumps over the lazy dog.",
                    "title": "Object0"
                }
            ]
        }
    }
}


In [18]:
# Generative natural language output
# https://weaviate.io/developers/weaviate/quickstart#generative-search-grouped-task
# I don't think ChatGPT is being used for the text generation. It could be some other OpenAi model.

query_text1 = "Fox"
query_text2 = "Write a poem about the dog"

response = (
    client.query
    .get("MyTable1", ["title", "foo", "quote_text"])
    .with_near_text({"concepts": [query_text1]})
    .with_generate(grouped_task=query_text2)
    .with_limit(2)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "MyTable1": [
                {
                    "_additional": {
                        "distance": 0.20118356,
                        "generate": {
                            "error": null,
                            "groupedResult": "In a world where foxes dance with grace,\nA sleeping hound finds its resting place.\nA nimble red fox, swift and sly,\nLeaps over the hound with a gleam in its eye.\n\nWith elegance and poise, it takes to the air,\nA breathtaking sight, beyond compare.\nThe hound, unaware, dreams of chasing its prey,\nWhile the fox, in a moment, steals away.\n\nBut in another tale, a different scene,\nThe lazy dog, a sight to be seen.\nThe quick brown fox, with a playful bound,\nJumps over the dog, without making a sound.\n\nThe dog, unmoving, lies in repose,\nAs the fox frolics, its energy flows.\nA testament to the dog's tranquil state,\nAs the fox dances, it cannot abate.\n\nTwo stories intertwined, both unique,\nTh

In [19]:
# Search one row against other rows

response = (
    client.query
    .get("MyTable1", ["title", "foo", "quote_text"])
    .with_near_object({ # the id of the the search object
        "id": "1cf7fdad-30b3-434f-80d6-eeb5a72e7784" # "The quick brown fox jumps over the lazy dog."
    })
    .with_limit(3)
    .with_additional(["distance"])
    .do()
)

#result = response["data"]["Get"]["MyCollection"]
#print(json.dumps(result, indent=2))

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "MyTable1": null
        }
    },
    "errors": [
        {
            "locations": [
                {
                    "column": 6,
                    "line": 1
                }
            ],
            "message": "explorer: get class: vectorize params: nearObject params: vector not found",
            "path": [
                "Get",
                "MyTable1"
            ]
        }
    ]
}


In [20]:
# It's also possible to search with filters

"""
response = (
    client.query
    .get("MyCollection", ["title", "foo"])
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
    })
    .with_additional(["distance, id"]) # output the distance of the query vector to the objects in the database
    .with_where({
        "path": ["foo"],
        "operator": "GreaterThan",
        "valueNumber": 44
    })
    .with_limit(2) # limit the output to only 2
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))
"""

'\nresponse = (\n    client.query\n    .get("MyCollection", ["title", "foo"])\n    .with_near_vector({\n        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]\n    })\n    .with_additional(["distance, id"]) # output the distance of the query vector to the objects in the database\n    .with_where({\n        "path": ["foo"],\n        "operator": "GreaterThan",\n        "valueNumber": 44\n    })\n    .with_limit(2) # limit the output to only 2\n    .do()\n)\n\nresult = response["data"]["Get"]["MyCollection"]\nprint(json.dumps(result, indent=2))\n'